In [1]:
# Download the data
!mkdir oskar
!cd oskar
!curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.2/Visium_HD_Human_Kidney_FFPE/Visium_HD_Human_Kidney_FFPE_web_summary.html
!curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.2/Visium_HD_Human_Kidney_FFPE/Visium_HD_Human_Kidney_FFPE_spatial.tar.gz
!curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.2/Visium_HD_Human_Kidney_FFPE/Visium_HD_Human_Kidney_FFPE_binned_outputs.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.3M    0 12.3M    0     0  23.2M      0 --:--:-- --:--:-- --:--:-- 23.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3175M  100 3175M    0     0  76.2M      0  0:00:41  0:00:41 --:--:-- 70.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  480M  100  480M    0     0  24.2M      0  0:00:19  0:00:19 --:--:-- 26.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1811  100  1811    0     0   2536      0 --:--:-- --:--:-- --:--:--  2536
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
!tar -xzvf Visium_HD_Human_Kidney_FFPE_binned_outputs.tar.gz

binned_outputs/
binned_outputs/square_002um/
binned_outputs/square_002um/filtered_feature_bc_matrix/
binned_outputs/square_002um/filtered_feature_bc_matrix/matrix.mtx.gz
binned_outputs/square_002um/filtered_feature_bc_matrix/features.tsv.gz
binned_outputs/square_002um/filtered_feature_bc_matrix/barcodes.tsv.gz
binned_outputs/square_002um/filtered_feature_bc_matrix.h5
binned_outputs/square_002um/raw_feature_bc_matrix/
binned_outputs/square_002um/raw_feature_bc_matrix/matrix.mtx.gz
binned_outputs/square_002um/raw_feature_bc_matrix/features.tsv.gz
binned_outputs/square_002um/raw_feature_bc_matrix/barcodes.tsv.gz
binned_outputs/square_002um/raw_feature_bc_matrix.h5
binned_outputs/square_002um/raw_probe_bc_matrix.h5
binned_outputs/square_002um/spatial/
binned_outputs/square_002um/spatial/aligned_fiducials.jpg
binned_outputs/square_002um/spatial/aligned_tissue_image.jpg
binned_outputs/square_002um/spatial/cytassist_image.tiff
binned_outputs/square_002um/spatial/detected_tissue_image.jpg
binn

In [6]:
!tar -xzvf Visium_HD_Human_Kidney_FFPE_spatial.tar.gz

spatial/
spatial/aligned_fiducials.jpg
spatial/aligned_tissue_image.jpg
spatial/cytassist_image.tiff
spatial/detected_tissue_image.jpg
spatial/tissue_hires_image.png
spatial/tissue_lowres_image.png


In [ ]:
cd ..

In [1]:
# read the 16um h5 file
import pandas as pd
import scanpy as sc
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from itertools import product
from shapely import Polygon, MultiPoint
from scipy.optimize import minimize


#read expression object for 16um

adata_16um = sc.read_10x_h5('oskar/binned_outputs/square_016um/filtered_feature_bc_matrix.h5')


/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
# read cluster annotations for 16um
clusters = pd.read_csv('oskar/binned_outputs/square_016um/analysis/clustering/gene_expression_graphclust/clusters.csv')

In [4]:
# add cluster annotations to adata.obs
adata_16um.obs = adata_16um.obs.reset_index().rename(columns = {'index':'Barcode'}).merge(clusters, on = 'Barcode' , how = 'left')

In [5]:
adata_16um.obs = adata_16um.obs.assign(Cluster = adata_16um.obs.Cluster.astype(str))

In [6]:
# only keep the cluster 5
adata_16um = adata_16um[adata_16um.obs.Cluster=='5'].copy()

/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [7]:
obs = adata_16um.obs

In [8]:
obs = obs.rename(columns = {'Barcode':'barcode'})

In [9]:
# fit a grid and add 16um spots as square shapes  in geopandas dataframe 

tissue_pos_parquet_path = 'oskar/binned_outputs/square_016um/spatial/tissue_positions.parquet'
def convert_obs_to_gdf(obs_orig, tissue_pos_parquet_path, initial_guess = [0,0,10,0], correlation_sign=+1):
    if tissue_pos_parquet_path is not None:
        tissue_pos= pd.read_parquet(tissue_pos_parquet_path)               
        obs = obs_orig.merge(tissue_pos, on='barcode')
    else:
        obs = obs_orig.copy()
    if correlation_sign *(abs(np.corrcoef(obs.pxl_col_in_fullres, obs.array_row)[0,1])-abs(np.corrcoef(obs.pxl_col_in_fullres, obs.array_col)[0,1]))<0:           
        obs = obs.assign(grid_col = obs.array_row)                                                                                          
        obs = obs.assign(grid_row = obs.array_col)                                                                                          
    else:                                                                                                                                   
        obs = obs.assign(grid_col = obs.array_col)                                                                                          
        obs = obs.assign(grid_row = obs.array_row)                                                                                          
    x = obs.grid_col                                                                                                                        
    y = obs.grid_row                                                                                                                        
    X = obs.pxl_col_in_fullres                                                                                                              
    Y = obs.pxl_row_in_fullres                                                                                                              
    print(np.corrcoef(obs.pxl_col_in_fullres, obs.grid_col))                                                                                
    def objective(params):                                                                                                                  
        X0, Y0, c, theta = params                                                                                                           
        # Predicted X and Y values                                                                                                          
        X_pred = X0 + c * x * np.cos(theta) - c * y * np.sin(theta)                                                                         
        Y_pred = Y0 + c * y * np.cos(theta) + c * x * np.sin(theta)                                                                         
        # Residuals                                                                                                                         
        residuals_X = X - X_pred                                                                                                            
        residuals_Y = Y - Y_pred                                                                                                            
        # Sum of squared residuals                                                                                                          
        return np.sum(residuals_X**2 + residuals_Y**2)                                                                                      
    result = minimize(objective, initial_guess, method='BFGS')                                                                              
    X0, Y0, c, theta = result.x                                                                                                             
                                                                                                                                            
    print(f"X0: {X0}")                                                                                                                      
    print(f"Y0: {Y0}")                                                                                                                      
    print(f"c: {c}")                                                                                                                        
    print(f"Theta (radians): {theta}")                                                                                                      
    def transform(x, y):                                                                                                                    
        return (X0 + c * x * np.cos(theta) - c * y * np.sin(theta),  Y0 + c * y * np.cos(theta) + c * x * np.sin(theta))                    
    def reverse_transform(x_prime, y_prime):                                                                                                
        # Constants                                                                                                                         
        denominator = c
     
        # Compute x and y
        x = ((x_prime - X0) * np.cos(theta) + (y_prime - Y0) * np.sin(theta)) / denominator
        y = ((y_prime - Y0) * np.cos(theta) - (x_prime - X0) * np.sin(theta)) / denominator
     
        return x, y

    def compute_polygon(row):
        # Compute the four vertices using the transform function
        v1 = transform(row['grid_col'] - 0.5, row['grid_row'] - 0.5)
        v2 = transform(row['grid_col'] + 0.5, row['grid_row'] - 0.5)
        v3 = transform(row['grid_col'] + 0.5, row['grid_row'] + 0.5)
        v4 = transform(row['grid_col'] - 0.5, row['grid_row'] + 0.5)
        # Create a polygon from the vertices
        return Polygon([v1, v2, v3, v4])
    obs = obs.assign(geometry = obs.apply(compute_polygon, axis=1))
    return gpd.GeoDataFrame(obs, geometry='geometry')
gobs = convert_obs_to_gdf(obs, tissue_pos_parquet_path)


# merge neighboring spots into bigger shapes (Capsules)
gdf = gobs.loc[:,'Cluster geometry'.split()].dissolve(by='Cluster').reset_index().explode().reset_index().rename(columns = {'index':'ID'})


[[1.         0.10591709]
 [0.10591709 1.        ]]
X0: 27601.792643404482
Y0: 12547.387793882503
c: 58.442573416754065
Theta (radians): 1.5660286061453217


In [12]:
gdf = gdf.assign(ID = [i for i in range(len(gdf))])

In [15]:
# functions to intersect geopandas dataframes


def assign_shape_id(obs, gdf):
    """
    Assigns a shape_id to each shape in obs based on the gdf shape with the most shared area.

    Parameters:
    obs (GeoDataFrame): The GeoDataFrame with smaller shapes (8000 shapes).
    gdf (GeoDataFrame): The GeoDataFrame with larger shapes (161 shapes) and an 'ID' column.

    Returns:
    GeoDataFrame: The obs GeoDataFrame with a new column 'shape_id'.
    """
    # Perform a spatial overlay to get intersections
    intersections = gpd.overlay(obs, gdf, how='intersection')

    # Calculate the area of intersection for each pair
    intersections['area'] = intersections.geometry.area

    # Find the shape in gdf that shares the most area with each shape in obs
    max_area_idx = intersections.groupby(obs.index)['area'].idxmax()

    # Extract the corresponding ID from gdf
    obs['shape_id'] = intersections.loc[max_area_idx, 'ID'].values

    return obs


def assign_shape_id_contain(obs, gdf):
    """
    Assigns a shape_id to each shape in obs based on the gdf shape that CONTAINS it.

    Parameters:
    obs (GeoDataFrame): smaller shapes
    gdf (GeoDataFrame): larger shapes with an 'ID' column

    Returns:
    GeoDataFrame: obs with new column 'shape_id'
    """
    # Make sure same CRS
    if obs.crs != gdf.crs:
        gdf = gdf.to_crs(obs.crs)

    # Spatial join: keep all obs rows, attach ID of polygon that contains them
    joined = gpd.sjoin(
        obs,
        gdf[['ID', 'geometry']],
        how='left',
        predicate='within',   # or 'contains' if you swap obs/gdf sides
    )

    # joined has all obs columns + ID; just rename to shape_id
    joined = joined.rename(columns={'ID': 'shape_id'})

    # Drop sjoin helper columns if you don’t want them
    joined = joined.drop(columns=[c for c in joined.columns if c.startswith('index_')], errors='ignore')

    return joined



In [16]:
# read 2um expression
adata_2um = sc.read_10x_h5('oskar/binned_outputs/square_002um/filtered_feature_bc_matrix.h5')


/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [17]:
#read 2u positions file
obs_2um = pd.read_parquet('oskar/binned_outputs/square_002um/spatial/tissue_positions.parquet')

In [18]:
# use the center points of the 2um spots to define shapes and only retain the ones that fall in side a capsule
obs_2um = gpd.GeoDataFrame(
    obs_2um,
    geometry=gpd.points_from_xy(
        obs_2um["pxl_col_in_fullres"],  # x
        obs_2um["pxl_row_in_fullres"],  # y
    ),
    crs=None,  # or set a CRS if these are in some real-world coordinate system
)


In [19]:
obs_2um_filtered = assign_shape_id_contain(obs_2um, gdf)

In [20]:
obs_2um_filtered = obs_2um_filtered[~pd.isna(obs_2um_filtered.shape_id)]

In [22]:
# make 2um spots as square shapes, uncomment this if you want to plot them 
#obs_2um_spots = convert_obs_to_gdf(obs_2um_filtered, None)

In [38]:
#tst = obs_2um_spots[obs_2um_spots.shape_id==2]

In [23]:

# def write_qp_geojson_AB(
#     gdf: gpd.GeoDataFrame,
#     output_geojson_path: str,
#     p: float = 0.5,
#     label_column: str = "group"
# ):
#     """
#     Randomly assign shapes in `gdf` to 'A' and 'B' using a Binomial(n, p),
#     and write a GeoJSON that QuPath will interpret as classifications.

#     Parameters
#     ----------
#     gdf : GeoDataFrame
#         Input geometries (pixel coordinates for QuPath).
#     output_geojson_path : str
#         Path to the output GeoJSON file.
#     p : float, optional
#         Probability that a given shape is labeled 'A'.
#         The number of 'A' labels is drawn from Binomial(n, p).
#     label_column : str, optional
#         Name of the column to store 'A'/'B' group labels.
#     """
#     if not isinstance(gdf, gpd.GeoDataFrame):
#         raise TypeError("gdf must be a GeoDataFrame")

#     if not (0.0 <= p <= 1.0):
#         raise ValueError("p must be between 0 and 1")

#     gdf_out = gdf.copy()
#     n = len(gdf_out)
#     if n == 0:
#         raise ValueError("gdf has no rows")

#     # --- 1) Draw number of A's from Binomial(n, p) ---
#     n_A = np.random.binomial(n, p)

#     # --- 2) Create labels and randomly assign to rows ---
#     labels = np.array(["A"] * n_A + ["B"] * (n - n_A))
#     perm = np.random.permutation(n)

#     gdf_out[label_column] = None
#     gdf_out.loc[gdf_out.index[perm], label_column] = labels

#     # --- 3) Build QuPath-compatible 'classification' property ---
#     # This becomes properties.classification.name = "A"/"B"
#     gdf_out["classification"] = gdf_out[label_column]

#     # Optional: mark as annotations for QuPath
#     gdf_out["object_type"] = "annotation"

#     # --- 4) Write GeoJSON ---
#     gdf_out.to_file(output_geojson_path, driver="GeoJSON")

#     return gdf_out


In [45]:
#gdf_out = write_qp_geojson_AB(tst,'oskar/sample_capsule_2um_split.geojson')

/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [47]:
#write_qp_geojson_AB(tst,'oskar/sample_capsule_2um_split.geojson')
#write_qp_geojson_AB(tst,'oskar/sample_capsule_2um_split_0poit3.geojson', p=0.3)
#write_qp_geojson_AB(tst,'oskar/sample_capsule_2um_split_0poit1.geojson', p=0.1)


/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


,barcode,in_tissue,array_row,array_col,pxl_row_in_fullres,pxl_col_in_fullres,geometry,shape_id,grid_col,grid_row,group,classification,object_type
4291264,s_002um_01280_03264-1,1,1280,3264,21985.635230,3826.493489,"POLYGON ((3831.156 21982.417, 3831.191 21989.7...",2.0,1280,3264,B,B,annotation
4291265,s_002um_01280_03265-1,1,1280,3265,21985.669695,3819.187737,"POLYGON ((3823.851 21982.452, 3823.886 21989.7...",2.0,1280,3265,B,B,annotation
4291266,s_002um_01280_03266-1,1,1280,3266,21985.704160,3811.881986,"POLYGON ((3816.546 21982.487, 3816.58 21989.79...",2.0,1280,3266,B,B,annotation
4291267,s_002um_01280_03267-1,1,1280,3267,21985.738625,3804.576235,"POLYGON ((3809.24 21982.522, 3809.275 21989.82...",2.0,1280,3267,B,B,annotation
4291268,s_002um_01280_03268-1,1,1280,3268,21985.773091,3797.270484,"POLYGON ((3801.935 21982.557, 3801.97 21989.86...",2.0,1280,3268,A,A,annotation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448741,s_002um_01327_03291-1,1,1327,3291,22329.883306,3630.886858,"POLYGON ((3635.552 22326.704, 3635.586 22334.0...",2.0,1327,3291,A,A,annotation
4448742,s_002um_01327_03292-1,1,1327,3292,22329.917770,3623.581108,"POLYGON ((3628.246 22326.739, 3628.281 22334.0...",2.0,1327,3292,B,B,annotation
4448743,s_002um_01327_03293-1,1,1327,3293,22329.952235,3616.275358,"POLYGON ((3620.941 22326.774, 3620.976 22334.0...",2.0,1327,3293,B,B,annotation
4448744,s_002um_01327_03294-1,1,1327,3294,22329.986699,3608.969608,"POLYGON ((3613.636 22326.808, 3613.671 22334.1...",2.0,1327,3294,B,B,annotation


In [24]:
adata_2um = adata_2um[adata_2um.obs.index.isin(set(obs_2um_filtered.barcode))].copy()

/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [25]:
adata_2um.obs = adata_2um.obs.reset_index().rename(columns={'index':'barcode'}).merge(obs_2um_filtered, on='barcode', how='left')

In [26]:
adata_2um.obs = adata_2um.obs.drop(columns='geometry')

In [43]:
adata_2um.write('oskar/podocytes_2um.h5ad')

SyntaxError: invalid syntax (3127486080.py, line 1)

In [44]:
gobs

,barcode,Cluster,in_tissue,array_row,array_col,pxl_row_in_fullres,pxl_col_in_fullres,grid_col,grid_row,geometry
0,s_016um_00107_00066-1,5,1,107,66,18819.845848,23774.814585,107,66,"POLYGON ((23803.522 18789.702, 23803.801 18848..."
1,s_016um_00199_00064-1,5,1,199,64,24195.528524,23917.513069,199,64,"POLYGON ((23946.041 24165.8, 23946.319 24224.2..."
2,s_016um_00074_00316-1,5,1,74,316,16960.361870,9153.955503,74,316,"POLYGON ((9183.85 16930.778, 9184.129 16989.22..."
3,s_016um_00151_00152-1,5,1,151,152,21414.801969,18760.755280,151,152,"POLYGON ((18789.778 21385.109, 18790.057 21443..."
4,s_016um_00343_00060-1,5,1,343,60,32609.388920,24191.689975,343,60,"POLYGON ((24219.932 32580.321, 24220.211 32638..."
...,...,...,...,...,...,...,...,...,...,...
8541,s_016um_00037_00017-1,5,1,37,17,14715.713082,26619.067414,37,17,"POLYGON ((26647.671 14685.115, 26647.95 14743...."
8542,s_016um_00089_00123-1,5,1,89,123,17783.692992,20438.311187,89,123,"POLYGON ((20467.318 17753.63, 20467.597 17812...."
8543,s_016um_00029_00063-1,5,1,29,63,14260.901925,23928.276066,29,63,"POLYGON ((23957.114 14230.397, 23957.393 14288..."
8544,s_016um_00229_00203-1,5,1,229,203,25986.964472,15801.870916,229,203,"POLYGON ((15830.975 25957.788, 15831.253 26016..."


In [46]:
gobs = assign_shape_id(gobs, gdf)

/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 20 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


In [34]:
adata = adata_16um.copy()

/home/hosein.toosi/ovary/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [57]:
adata.obs = adata.obs.rename(columns = {'Barcode':'barcode'}).merge(gobs, on ='barcode Cluster'.split(), how ='left')

In [35]:
#plotting functions, uncomment if you want to plot
# import geopandas as gpd
# import matplotlib.pyplot as plt
# import numpy as np
# from shapely.affinity import scale as shp_scale
# from matplotlib.image import imread

# def plot_scaled_gdf_on_image(gdf: gpd.GeoDataFrame,
#                              image,
#                              scale_factor: float,
#                              output_png_path: str,
#                              line_color: str = "green",
#                              line_width: float = 1.0):
#     """
#     Scale a copy of `gdf` about the origin (0,0), overlay it on `image`,
#     and save to `output_png_path` as PNG.

#     Parameters
#     ----------
#     gdf : GeoDataFrame
#         Input geometries (assumed to be in same coordinate space you want to plot).
#     image : str | numpy.ndarray
#         Path to an image file or a NumPy array (H×W×C) already loaded.
#     scale_factor : float
#         Uniform scale factor applied to x and y about the origin (0, 0).
#     output_png_path : str
#         Where to save the resulting PNG.
#     line_color : str, optional
#         Color of the geometry outlines. Default "green".
#     line_width : float, optional
#         Line width for outlines. Default 1.0.
#     """
#     if not isinstance(gdf, gpd.GeoDataFrame):
#         raise TypeError("gdf must be a GeoDataFrame")

#     # 1) Scale a copy of the geometries about the origin (0,0)
#     gdf_scaled = gdf.copy()
#     gdf_scaled["geometry"] = gdf_scaled["geometry"].apply(
#         lambda geom: shp_scale(geom, xfact=scale_factor, yfact=scale_factor, origin=(0, 0))
#     )

#     # 2) Load the background image (path or array)
#     if isinstance(image, str):
#         img = imread(image)
#     elif isinstance(image, np.ndarray):
#         img = image
#     else:
#         raise TypeError("image must be a file path or a NumPy array")

#     # 3) Decide the extent to place the image.
#     #    Here we pin the image to the ORIGINAL (unscaled) gdf bounds so the image
#     #    stays fixed while the geometry scales relative to the coordinate origin.
#     #minx, miny, maxx, maxy = gdf.total_bounds
#     #extent = (minx, maxx, miny, maxy)

#     # 4) Plot
#     fig, ax = plt.subplots(figsize=(20,20))
#     ax.imshow(img, origin="upper")   # image anchored to original bounds
#     gdf_scaled.boundary.plot(ax=ax, color=line_color, linewidth=line_width)

#     ax.set_aspect("equal")
#     # ax.set_xlim(minx, maxx)
#     # ax.set_ylim(miny, maxy)
#     ax.set_xlabel("X")
#     ax.set_ylabel("Y")
#     ax.set_title(f"Scaled geometries (factor={scale_factor}) over image")

#     # 5) Save and clean up
#     plt.savefig(output_png_path, dpi=300, bbox_inches="tight")
#     plt.close(fig)


In [13]:
#plot_scaled_gdf_on_image(gdf=gdf, scale_factor=0.13636674, image='oskar/spatial/tissue_hires_image.png', output_png_path='Podocytes_outline_new.png', line_width=0.5, line_color='yellow')

In [58]:
import anndata
import numpy as np
import pandas as pd

def aggregate_by_shape_id(adata):
    """
    Aggregates an AnnData object by summing up observations with the same shape_id.

    Parameters:
    adata (AnnData): The input AnnData object with a column 'shape_id' in adata.obs.

    Returns:
    AnnData: A new AnnData object with aggregated data.
    """
    # Ensure shape_id is treated as a categorical variable
    adata.obs['shape_id'] = adata.obs['shape_id'].astype(str)

    # Convert to DataFrame for aggregation
    X_df = pd.DataFrame(adata.X.todense(), index=adata.obs.index)
    
    # Group by shape_id and sum the counts
    grouped_X = X_df.groupby(adata.obs['shape_id']).sum()


    # Create new AnnData object
    ndata = anndata.AnnData(X=grouped_X.values)
    
    # Add shape_id as an index to obs
    ndata.obs = pd.DataFrame(index=grouped_X.index)
    ndata.var = adata.var
    
    return ndata

# Example usage:
# ndata = aggregate_by_shape_id(adata)

# Display the new AnnData object
# tools.display_dataframe_to_user(name="Aggregated Data", dataframe=pd.DataFrame(ndata.X, index=ndata.obs.index))


In [59]:
ndata = aggregate_by_shape_id(adata)

In [60]:
ndata

AnnData object with n_obs × n_vars = 161 × 18085
    var: 'gene_ids', 'feature_types', 'genome'

In [62]:
ndata.write_h5ad('merged_blobs_in_cluster_5.h5ad')